# Imports

In [ ]:
import calmap
import folium
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pyproj import Transformer

# Load and clean data

In [ ]:
df = pd.read_csv("traffic_data.csv", sep=";", encoding="utf-8")

# Convert date to datetime
df["date"] = pd.to_datetime(df["date"], format="%d.%m.%Y")

# Clean hourly columns
hour_cols = [f"{i:02d}_{i+1:02d}" for i in range(24)]
df[hour_cols] = df[hour_cols].replace(" ", "", regex=True).apply(pd.to_numeric, errors='coerce')

## Data overview

In [ ]:
mem_mb = df.memory_usage(deep=True).sum() / 1024 ** 2
print(f"📦 Total data size: {mem_mb:.2f} MB")
print(f"🔢 Rows: {df.shape[0]}, Columns: {df.shape[1]}")

df["date"] = pd.to_datetime(df["date"], format="%d.%m.%Y", errors="coerce")
print(f"📅 Date range: {df['date'].min().date()} → {df['date'].max().date()}")
print("\n🧾 Columns:\n", df.columns.tolist())

# Check for missing values
print("\n❓ Missing values:\n", df.isna().sum())

hour_cols = [f"{i:02d}_{i+1:02d}" for i in range(24)]
df[hour_cols] = df[hour_cols].replace(" ", "", regex=True).apply(pd.to_numeric, errors="coerce")
df["daily_total"] = df[hour_cols].sum(axis=1)

plt.figure(figsize=(8, 4))
sns.histplot(df["daily_total"].dropna(), bins=50, kde=True)
plt.title("📈 Distribution of Daily Total Traffic")
plt.xlabel("Total Vehicles per Day")
plt.tight_layout()
plt.show()

hourly_means = df[hour_cols].mean()

plt.figure(figsize=(10, 4))
sns.lineplot(x=range(24), y=hourly_means)
plt.xticks(range(24), [f"{i:02d}:00" for i in range(24)], rotation=45)
plt.title("🕒 Average Traffic by Hour")
plt.ylabel("Average Vehicles")
plt.xlabel("Hour of Day")
plt.tight_layout()
plt.show()

# Mocks

## Daily traffic profiles

In [ ]:
selected_dates = df["date"].unique()[:3]  # First 3 dates
fig, ax = plt.subplots(figsize=(10, 6))

for day in selected_dates:
    day_data = df[df["date"] == day][hour_cols].sum()
    ax.plot(range(24), day_data, label=day.strftime("%Y-%m-%d"))

ax.set_xticks(range(24))
ax.set_xticklabels([f"{i:02d}:00" for i in range(24)], rotation=45)
ax.set_ylabel("Traffic count")
ax.set_title("Daily Traffic Profiles")
ax.legend()
plt.tight_layout()
plt.show()

## Rolling weekly trends of daily traffic

In [ ]:
df["daily_total"] = df[hour_cols].sum(axis=1)
daily_traffic = df.groupby("date")["daily_total"].sum().sort_index()
rolling = daily_traffic.rolling(7).mean()

fig, ax = plt.subplots(figsize=(12, 6))
daily_traffic.plot(ax=ax, label="Daily Total", alpha=0.5)
rolling.plot(ax=ax, label="7-Day Rolling Average", linewidth=2)
ax.set_title("Rolling Weekly Trends of Daily Traffic")
ax.set_ylabel("Total Vehicles")
ax.legend()
plt.tight_layout()
plt.show()

## Calendar heatmap of daily traffic

In [ ]:
plt.figure(figsize=(12, 6))
calmap.calendarplot(daily_traffic, how='sum', cmap='YlOrRd', fillcolor='lightgrey', linewidth=0.5)
plt.suptitle("Calendar Heatmap of Daily Traffic", fontsize=16)
plt.tight_layout()
plt.show()

## Copenhagen traffic map

In [ ]:
df["avg_daily_traffic"] = df[hour_cols].sum(axis=1)
location_agg = df.groupby(["x_coord", "y_coord"]).agg({
    "avg_daily_traffic": "mean",
    "street_name": "first"
}).reset_index()

transformer = Transformer.from_crs("epsg:25832", "epsg:4326", always_xy=True)  # UTM zone 32N -> WGS84
location_agg["lon"], location_agg["lat"] = transformer.transform(
    location_agg["x_coord"].values,
    location_agg["y_coord"].values
)

m = folium.Map(location=[55.6761, 12.5683], zoom_start=12)  # Centered on Copenhagen

# Add points
for _, row in location_agg.iterrows():
    folium.CircleMarker(
        location=(row["lat"], row["lon"]),
        radius=max(row["avg_daily_traffic"] / 500, 3),  # scale radius
        color="blue",
        fill=True,
        fill_opacity=0.6,
        popup=f"{row['street_name']}: {int(row['avg_daily_traffic'])} vehicles/day"
    ).add_to(m)

# Save to HTML
m.save("copenhagen_traffic_map.html")
print("✅ Map saved to 'copenhagen_traffic_map.html'")